### Summarization patterns with langchain library
Adapted from langchain's documentation at https://python.langchain.com/en/latest/modules/chains/index_examples/summarize.html

Do pip install ```pip install langchain``` and fill in your information below

In [52]:
from azure.identity import InteractiveBrowserCredential
import os

import openai
interactive_credential = InteractiveBrowserCredential() # add tenant_id="YOUR_TENANT_ID" if you have more than 1 tenants
token = interactive_credential.get_token("https://cognitiveservices.azure.com/.default")

In [42]:
from langchain.chat_models.azure_openai import AzureChatOpenAI
from langchain.agents.react.base import DocstoreExplorer
GPT_ENGINE = "gpt-35-turbo"
os.environ["LANGCHAIN_HANDLER"] = "langchain"
os.environ["OPENAI_API_KEY"] = token.token # replace with API key in case you don't use Azure AAD
os.environ["OPENAI_API_TYPE"] = "azure_ad" # replace with "azure" in case you don't use Azure AAD
service_name = "YOUR_SERVICE_NAME"
os.environ["OPENAI_API_BASE"] = f"https://{service_name}.openai.azure.com/"
os.environ["OPENAI_API_VERSION"] = "2023-03-15-preview"
llm= AzureChatOpenAI(deployment_name=GPT_ENGINE,temperature =0)

from langchain import OpenAI, PromptTemplate, LLMChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate
text_splitter = CharacterTextSplitter()

In [55]:
with open("state_of_the_union.txt", encoding='utf-8') as f:
    state_of_the_union = f.read()
texts = text_splitter.split_text(state_of_the_union)

In [56]:
from langchain.docstore.document import Document
from langchain.chains.summarize import load_summarize_chain

docs = [Document(page_content=t) for t in texts[:3]]

##### With map reduce pattern

In [57]:
chain = load_summarize_chain(llm, chain_type="map_reduce")
chain.run(docs)

"President Biden addressed Congress and the nation, emphasizing the importance of American diplomacy and resolve in standing with Ukraine and its allies against Russian aggression. The US is taking robust action to target Russia's economy with sanctions and providing military, economic, and humanitarian assistance to Ukraine. The President also highlighted his administration's new economic vision for America, including investing in infrastructure, education, and growing the workforce from the bottom up, and announced the passage of the Bipartisan Infrastructure Law."

##### With map stuff pattern

In [ ]:
from langchain.chains.summarize import load_summarize_chain
chain = load_summarize_chain(llm, chain_type="stuff")

chain.run(docs)


##### With refine pattern

In [ ]:
chain = load_summarize_chain(llm, chain_type="refine")

print(chain.run(docs))

In [ ]:
chain = load_summarize_chain(llm, chain_type="refine", return_intermediate_steps=True)

chain({"input_documents": docs}, return_only_outputs=True)


##### Customize prompt 

In [ ]:
prompt_template = """Write a concise summary of the following:


{text}


CONCISE SUMMARY:"""
PROMPT = PromptTemplate(template=prompt_template, input_variables=["text"])
refine_template = (
    "Your job is to produce a final summary\n"
    "We have provided an existing summary up to a certain point: {existing_answer}\n"
    "We have the opportunity to refine the existing summary"
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{text}\n"
    "------------\n"
    "Given the new context, refine the original summary. Make sure to address the list of problems, list of solutions and any following action"
    "If the context isn't useful, return the original summary."
)
refine_prompt = PromptTemplate(
    input_variables=["existing_answer", "text"],
    template=refine_template,
)
chain = load_summarize_chain(llm, chain_type="refine", return_intermediate_steps=True, question_prompt=PROMPT, refine_prompt=refine_prompt)
chain({"input_documents": docs}, return_only_outputs=True)